In [28]:

using Pkg;
Pkg.add(url="https://github.com/aicenter/GenerativeModels.jl", rev="compathelper/new_version/2021-03-29-00-05-30-904-1989780142")

     Cloning git-repo `https://github.com/aicenter/GenerativeModels.jl`
    Updating git-repo `https://github.com/aicenter/GenerativeModels.jl`
   Resolving package versions...
   Installed ConditionalDists ─ v0.4.10
   Installed StatsFuns ──────── v0.9.7
   Installed NNlibCUDA ──────── v0.2.1
   Installed NNlib ──────────── v0.8.2
   Installed Zygote ─────────── v0.6.12
   Installed ChainRules ─────── v0.7.70
   Installed StatsPlots ─────── v0.14.30
   Installed SpecialFunctions ─ v1.8.3
   Installed DistributionsAD ── v0.6.29
   Installed DiffRules ──────── v1.5.0
   Installed DrWatson ───────── v2.5.0
   Installed ChainRulesCore ─── v0.9.45
   Installed Plots ──────────── v1.25.5
   Installed Flux ───────────── v0.12.10
   Installed Reexport ───────── v0.2.0
   Installed FillArrays ─────── v0.11.9
   Installed LogExpFunctions ── v0.3.0
   Installed IPMeasures ─────── v0.2.4
   Installed Distributions ──── v0.24.18
   Installed Interpolations ─── v0.13.2
   Installed GR ─────────────

In [46]:
using DrWatson
quickactivate(@__DIR__)

In [33]:
Pkg.add("ConditionalDists")

   Resolving package versions...
    Updating `~/Swarm/Project.toml`
  [c648c4dd] + ConditionalDists v0.4.10
  No Changes to `~/Swarm/Manifest.toml`


In [53]:
using ConditionalDists: SplitLayer
using Flux, CSV,DataFrames,MLDataPattern

In [54]:
function normalise(M) 
    min = minimum(minimum(eachcol(M)))
    max = maximum(maximum(eachcol(M)))
    return (M .- min) ./ (max - min)
end

function load_and_normalise_data()
    df = DataFrame(CSV.File("$(datadir())/exp_raw/data_450.csv"; types=Float32))
    normalised = Array(df) |> normalise
    window_size = 60
    slidingwindow(normalised',window_size,stride=1)
end

data = load_and_normalise_data()
train_set, validate_set, test_set = splitobs(data, (0.7, 0.2));



In [40]:
function create_ae_1d()
    # Define the encoder and decoder networks 
    enc_map = Chain(
        # 60x900xb
        Conv((9,), 450 => 4500, relu; pad=SamePad()),
        MaxPool((2,)),
        # 30x4500xb
        Conv((5,), 4500 => 2250, relu; pad=SamePad()),
        MaxPool((2,)),
        # 15x2250xb
        Conv((3,), 2250 => 1000, relu; pad=SamePad()),
        Conv((3,), 1000 => 100, relu; pad=SamePad()),
        MaxPool((3,)),
        # 5x100xb
        Flux.flatten,
        SplitLayer( 500,[100,100],[identity, softplus])
    )
    encoder = ConditionalMvNormal(enc_map)
    
    dec_map = Chain(
        Dense(100, 500),
        (x -> reshape(x, 5, 100, :)),
        # 5x100xb
        ConvTranspose((3,), 100 => 1000, relu; pad=SamePad()),
        ConvTranspose((3,), 1000 => 2250, relu; pad=SamePad()),
        Upsample((3,)),
        # 15x2250xb
        ConvTranspose((5,), 2250 => 4500, relu; pad=SamePad()),
        Upsample((4,)),
        # 60x4500xb
        ConvTranspose((9,), 4500 => 450, relu; pad=SamePad()),
        # 60x450xb
        SplitLayer(60,[450,1],σ)
    ) 
    decoder = ConditionalMvNormal(dec_map)
    return VAE(100, encoder, decoder) |> gpu
end


create_ae_1d (generic function with 1 method)

In [41]:
model = create_ae_1d()

VAE:
 prior   = (DistributionsAD.TuringDiagMvNormal{CUDA.CuArray{Float32, 1, CUDA.Me...)
 encoder = (ConditionalMvNormal{Chain{Tuple{Conv{1, 2, typeof(relu), CUDA.CuArr...)
 decoder = (ConditionalMvNormal{Chain{Tuple{Dense{typeof(identity), CUDA.CuArra...)


In [42]:
loss(x) = -elbo(model,x)

loss (generic function with 1 method)

In [59]:
ps = Flux.params(model)
opt = ADAM()
d = train_set |> gpu

LoadError: Out of GPU memory trying to allocate 4.389 MiB
Effective GPU memory usage: 99.80% (11.760 GiB/11.784 GiB)
Memory pool usage: 9.778 GiB (9.781 GiB reserved)

In [58]:

for e in 1:5
#     @info "Epoch $e" loss(train_set)
    Flux.train!(loss, ps, d, opt)
end

LoadError: type DataType has no field mutable